In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [3]:
root_folder = "./tactile_dataset/"
file_name = "final_merged_df_sw500.csv"

df = pd.read_csv(root_folder + file_name)

In [4]:
X = df[['baro','imu_ax', 'imu_ay', 'imu_az','imu_gx', 'imu_gy', 'imu_gz','imu_mx', 'imu_my', 'imu_mz']].values
y = df['Texture']

print(f"Total samples collected: {X.shape[0]}")
print(f"Feature vector size: {X.shape[1]}")
print(f"Unique textures: {np.unique(y)}")

Total samples collected: 90665
Feature vector size: 10
Unique textures: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
texture_classes = label_encoder.classes_
num_classes = len(texture_classes)
print("\nEncoded texture labels:", texture_classes)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


Encoded texture labels: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [18]:
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)

class ANN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [19]:
model = ANN(10, 100, 12)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [20]:
num_epochs = 20

for epoch in range(num_epochs):
    accuracy = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        accuracy += (y_batch == y_pred.argmax(1)).sum().item()
    print(f"Epoch {epoch+1}, loss: {loss.item()}, accuracy: {accuracy/X_train.shape[0]}")

Epoch 1, loss: 1.441757082939148, accuracy: 0.49746318866155625
Epoch 2, loss: 1.1783651113510132, accuracy: 0.5661501130535488
Epoch 3, loss: 0.7587352991104126, accuracy: 0.5958611371532565
Epoch 4, loss: 0.7874339818954468, accuracy: 0.6200849280317653
Epoch 5, loss: 0.9099739789962769, accuracy: 0.639510836596261
Epoch 6, loss: 0.6036234498023987, accuracy: 0.6528428831412342
Epoch 7, loss: 0.9568380117416382, accuracy: 0.6635416092207578
Epoch 8, loss: 1.486978530883789, accuracy: 0.6713450614901009
Epoch 9, loss: 0.8184420466423035, accuracy: 0.6797413555396239
Epoch 10, loss: 1.3229458332061768, accuracy: 0.6874896597363922
Epoch 11, loss: 0.49721986055374146, accuracy: 0.691239728671483
Epoch 12, loss: 0.7134292721748352, accuracy: 0.6948243533888491
Epoch 13, loss: 0.6572558879852295, accuracy: 0.6982297468703469
Epoch 14, loss: 1.2584407329559326, accuracy: 0.702200408095737
Epoch 15, loss: 0.5426661372184753, accuracy: 0.7077014283350797
Epoch 16, loss: 0.787103533744812, ac